# Concurrencia y métricas de desempeño

Este cuaderno ensaya el conteo de objetos y registro de eventos (ingreso/egreso) con regiones definidas (asumiendo que la posición de la cámara está fija).

Salidas:
- Archivos CSV.
- Video 

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../videoanalytics/src")
import videoanalytics
videoanalytics.__version__

'0.0.2'

In [3]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib.pyplot'

## Chequeo de disponibilidad de HW / GPU

In [ ]:
!nvidia-smi

Detectadas por tensorflow en conda (FIXME).

In [1]:
import tensorflow as tf
print("GPUs disponibles: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

AttributeError: module 'tensorflow' has no attribute 'config'

## Datos de entrada y modelos

In [ ]:
WORKSPACE_PATH="../"
WORKSPACE_DATA_PATH=WORKSPACE_PATH+"/data/"
INPUT_VIDEO_PATH=WORKSPACE_DATA_PATH+"media/videos-youtube/"
!ls {INPUT_VIDEO_PATH}

Definición de ROIs.

In [ ]:
ROI_DEFINITION_FILE = WORKSPACE_DATA_PATH+"other/mirtar-rois.json"
!cat {ROI_DEFINITION_FILE}

Repositorio de modelos (local).

In [ ]:
MODELS_PATH=WORKSPACE_DATA_PATH+"models/"
!ls {MODELS_PATH}

In [ ]:
!tree {MODELS_PATH}/kaggle-fisheries-yolo

In [ ]:
!tree {MODELS_PATH}/mirtar-yolo

In [ ]:
!tree {MODELS_PATH}/feature_extractor_mars

## Definición de cadenas de procesamiento

### Conteo de objetos usando ROIs

In [ ]:
from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter, VariableCSVWriter

from videoanalytics.pipeline.sinks.object_detection import DetectionsAnnotator, DetectionsCSVWriter
from videoanalytics.pipeline.sinks.object_detection.yolo4 import YOLOv4DetectorTF

from videoanalytics.pipeline.sinks.roi import ROIView, ROIPresenceCounter
from videoanalytics.pipeline.sinks.visualization import TextOverlay

# ROIs

def make_object_counting_pipeline( input_video_filename,                                   
                                   output_det_csv_filename,
                                   output_var_csv_filename,
                                   output_video_filename,
                                   roi_definition_filename,
                                   detector_model_weights_filename,
                                   detector_model_classes_filename,                                   
                                   start_frame=0,
                                   max_frames=None):
    context = {}
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    
    # Detector
    pipeline.add_component( YOLOv4DetectorTF("detector",
                                             context,
                                             weights_filename=detector_model_weights_filename) )        
    pipeline.add_component( DetectionsCSVWriter("det_csv_writer",context,filename=output_det_csv_filename) )
    
    # ROIs view + contador
    pipeline.add_component( ROIView("roi_view",context,filename=roi_definition_filename,alpha=0.6 ) )
    pipeline.add_component( ROIPresenceCounter("roi_counter",context,filename=roi_definition_filename) )
    
    # Anotar detecciones en video
    pipeline.add_component( DetectionsAnnotator("annotator",context,
                                             class_names_filename=detector_model_classes_filename,
                                             show_label=True) )
        
    # Mostrar variables de contador de objetos en ROIs
    pipeline.add_component( TextOverlay("stats_text",context,
                                    text="Izq. (arriba): {q_izquierda_sup}\n"
                                         "Izq. (abajo): {q_izquierda_inf}\n"
                                         "Der. (arriba): {q_derecha_sup}\n"
                                         "Der. (abajo): {q_derecha_inf}\n"
                                         "Malacate: {q_malacate}\n"
                                         "Total: {q_total}",
                                    x=10,y=40) )

    # Almacenar variables en CSV
    pipeline.add_component( VariableCSVWriter("csv_variables",context,filename=output_var_csv_filename,
                                          variables_to_write=[
                                                "q_izquierda_sup",
                                                "q_izquierda_inf",
                                                "q_derecha_sup",
                                                "q_derecha_inf",
                                                "q_malacate",
                                                "q_total"
                                          ]))
    
    
    # Video de salida
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    pipeline.set_connections([
        ("input", "detector"),
        ("detector", "roi_counter"),
        ("detector", "det_csv_writer"),        
        ("detector", "roi_view"),
        ("roi_view", "annotator"),
        ("roi_counter", "csv_variables"),        
        ("csv_variables", "stats_text"),
        ("stats_text", "writer")
    ])
    
    return context,pipeline

### Registro de eventos usando ROIs

In [ ]:
DEEPSORT_MODEL_FILENAME = MODELS_PATH+"/feature_extractor_mars/mars-small128.pb"

In [ ]:
from videoanalytics.pipeline.sinks.object_tracking.deepsort import DeepSORT
from videoanalytics.pipeline.sinks.object_tracking import TrackedObjectsAnnotator, TrackedObjectsCSVWriter

from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter, VariableCSVWriter

from videoanalytics.pipeline.sinks.object_detection import DetectionsAnnotator, DetectionsCSVWriter
from videoanalytics.pipeline.sinks.object_detection.yolo4 import YOLOv4DetectorTF

from videoanalytics.pipeline.sinks.roi import ROIView, ROIPresenceCounter
from videoanalytics.pipeline.sinks.roi.events import ROIEvents, ROIEventsCSVWriter
from videoanalytics.pipeline.sinks.visualization import TextOverlay

def make_roi_events_pipeline( input_video_filename,                                   
                                   output_det_csv_filename,
                                   output_track_csv_filename,
                                   output_roievents_csv_filename,
                                   output_video_filename,
                                   roi_definition_filename,
                                   detector_model_weights_filename,
                                   detector_model_classes_filename,                                   
                                   start_frame=0,
                                   max_frames=None):
    context = {}
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    
    # Detector
    pipeline.add_component( YOLOv4DetectorTF("detector",
                                             context,
                                             weights_filename=detector_model_weights_filename) )        
    pipeline.add_component( DetectionsCSVWriter("det_csv_writer",context,filename=output_det_csv_filename) )
    
    # Tracker
    pipeline.add_component( DeepSORT("tracker", context, model_filename=DEEPSORT_MODEL_FILENAME) )
    pipeline.add_component( TrackedObjectsAnnotator("tracker_annotator",context) )
    pipeline.add_component( TrackedObjectsCSVWriter("tracker_csv_writer",context, 
                                                    filename = output_track_csv_filename) )
    
    # ROIs view + generación de eventos
    pipeline.add_component( ROIView("roi_view",context,filename=roi_definition_filename,alpha=0.6 ) )    
    pipeline.add_component( ROIEvents("roi_events",context,filename=roi_definition_filename) )

    # Registro de eventos a CSV
    pipeline.add_component( ROIEventsCSVWriter("roi_events_writer",context,output_roievents_csv_filename) )
    
    
    # Video de salida
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    pipeline.set_connections([
        ("input", "detector"),
        ("detector", "det_csv_writer"),
        ("detector", "tracker"),
        ("tracker", "tracker_csv_writer"),
        ("tracker", "roi_view"),
        ("roi_view", "tracker_annotator"),
        ("tracker", "roi_events"),
        ("roi_events", "roi_events_writer"),
        ("tracker_annotator", "writer")
    ])
                           
    return context,pipeline

## Batch job

Directorio de resultados.

In [ ]:
!ls {WORKSPACE_DATA_PATH}/output

In [ ]:
INPUT_VIDEOS = [
    (
        'buque EL MARISCO II ,proceso de captura merluz.mp4',
        None,
    ),
    
    (
        'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4',
        None,        
    ),
    
    (
        'Maniobra de pesca de merluza en mares argentino.mp4',
        None,
    ),
    
    ( 
        'maniobr.mp4',
        None
    ),
    
    ( 
        'mirtar-hk.mp4',
        None
    ),    
    
    (
        'Pesca artesanal de anchoita en mar del plat.mp4',
        None
    ),
    
    (
        'pesca de altura en el mar argentino,pesquero.mp4',
        None
    ),
    
    (
        'pesca de langostino,buque Alve.mp4',
        None
    ),
    
    (
        'Pesca de langostino en aguas nacionale.mp4',
        None
    ),
    
    (
       'pesca de langostino en rawson chubut con barco combenciona.mp4',
       None,
   ),
    
    (
        'PESCA DE LANGOSTINO RW CHUBU.mp4',
        None,
    ),
    
    (
        'PESCA EN RAWSO.mp4',
        None
    ),
    
    (
        'Un dia de pesca merluza  Buque Paola.mp4',
        None
    ),
    
    (
        'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4',
        None
    ),
    
    (
        'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4',
        None
    )
]

### Job para conteo de objetos por ROIs con Mirta-R (sin seguimiento)

- Detector entrenado con Mirtar-R.
- Nota: se aplica sólo al video que tiene ROIs etiquetadas.

In [ ]:
from os import path

def object_counting_batch_job(input_videos,output_path,dry_run=False):
    obj_det_model = "mirtar-yolo"
    t0 = datetime.now()
    print("Inicio de proceso: ", t0)
    
    for i,v in enumerate(input_videos):
        input_video_filename=v[0]        
        output_video_filename=output_path+"job{:d}.mp4".format(i)
        output_det_csv_filename=output_path+"job{:d}-detections.csv".format(i)
        output_var_csv_filename=output_path+"job{:d}-countings.csv".format(i)        

        print("Job {}. Procesando {}.".format(i, input_video_filename))
        
        if path.exists(output_video_filename):
            print("El archivo a generar {} ya existe. Pasando al siguiente".format(output_video_filename))    
            continue

        # FIXME, por ahora son fijos
        params=v[1]

        detector_model_weights_filename=MODELS_PATH+"/{}/checkpoints/yolo-416/".format(obj_det_model)
        detector_model_classes_filename=MODELS_PATH+"/{}/obj.names".format(obj_det_model)

        if not dry_run:
            
            context,pipeline = make_object_counting_pipeline( INPUT_VIDEO_PATH+input_video_filename,                                   
                                   output_det_csv_filename,
                                   output_var_csv_filename,
                                   output_video_filename,
                                   ROI_DEFINITION_FILE,
                                   detector_model_weights_filename,
                                   detector_model_classes_filename,                                   
                                   start_frame=0,
                                   max_frames=100)

            pipeline.execute()
            print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
            print("Video de salida guardado en {}".format(output_video_filename))
            
    t1 = datetime.now()
    print("Fin del proceso:",t1)        
    print("Tiempo transcurrido:",t1-t0)

In [ ]:
input_videos = INPUT_VIDEOS
output_base_path=WORKSPACE_DATA_PATH+"/output/"
obj_det_model = "mirtar-yolo"
output_path = output_base_path+obj_det_model+"-roi-objcount/"

In [ ]:
# Reset
#!rm -rf {output_path}
#!mkdir {output_path}

In [ ]:
object_counting_batch_job(input_videos,output_path,dry_run=False)

### Job para registro de eventos por ROIs con Mirta-R con seguimiento (DeepSORT)

- Detector entrenado con Mirtar-R.
- DeepSORT con MARS.
- Nota: se aplica sólo al video que tiene ROIs etiquetadas.

In [ ]:
from os import path

def roi_events_batch_job(input_videos,output_path,dry_run=False):
    obj_det_model = "mirtar-yolo"
    t0 = datetime.now()
    print("Inicio de proceso: ", t0)
    
    for i,v in enumerate(input_videos):
        input_video_filename=v[0]        
        output_video_filename=output_path+"job{:d}.mp4".format(i)
        output_det_csv_filename=output_path+"job{:d}-detections.csv".format(i)
        output_track_csv_filename=output_path+"job{:d}-trackings.csv".format(i)  
        output_roievents_csv_filename=output_path+"job{:d}-roievents.csv".format(i)  

        print("Job {}. Procesando {}.".format(i, input_video_filename))
        
        if path.exists(output_video_filename):
            print("El archivo a generar {} ya existe. Pasando al siguiente".format(output_video_filename))    
            continue

        # FIXME, por ahora son fijos
        params=v[1]

        detector_model_weights_filename=MODELS_PATH+"/{}/checkpoints/yolo-416/".format(obj_det_model)
        detector_model_classes_filename=MODELS_PATH+"/{}/obj.names".format(obj_det_model)

        if not dry_run:
            
            context,pipeline = make_roi_events_pipeline( INPUT_VIDEO_PATH+input_video_filename,
                                   output_det_csv_filename,
                                   output_track_csv_filename,
                                   output_roievents_csv_filename,
                                   output_video_filename,
                                   ROI_DEFINITION_FILE,
                                   detector_model_weights_filename,
                                   detector_model_classes_filename,                                   
                                   start_frame=0,
                                   max_frames=None)

            pipeline.execute()
            print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
            print("Video de salida guardado en {}".format(output_video_filename))
            
    t1 = datetime.now()
    print("Fin del proceso:",t1)        
    print("Tiempo transcurrido:",t1-t0)

In [ ]:
input_videos = INPUT_VIDEOS
output_base_path=WORKSPACE_DATA_PATH+"/output/"
obj_det_model = "mirtar-yolo"
output_path = output_base_path+obj_det_model+"-roi-events/"

In [ ]:
#Reset
#!rm -rf {output_path}
#!mkdir {output_path}

In [ ]:
roi_events_batch_job(input_videos,output_path,dry_run=False)

In [ ]:
class VideoProcessingApplication:

    def __init__(self,pipelines):
        pass
    
    def execute_parallel(self,max_jobs):
        